In [1]:
# autoreload import your package
%load_ext autoreload
%autoreload 2

from make_dataset import *


In [2]:
cfg = ExtractConfig(max_examples=(10, 10), max_length=999)
cfg


ExtractConfig(datasets=('amazon_polarity', 'super_glue:boolq', 'glue:qnli', 'imdb'), model='TheBloke/WizardCoder-Python-13B-V1.0-GPTQ', data_dirs=(), max_examples=(10, 10), num_shots=1, num_variants=-1, layers=(), seed=42, token_loc='last', template_path=None, max_length=999)

In [3]:
model, tokenizer = load_model(cfg.model)


2023-10-15 17:26:06.435 | INFO     | src.models.load:verbose_change_param:18 - changing pad_token_id from 32000 to 0
2023-10-15 17:26:06.435 | INFO     | src.models.load:verbose_change_param:18 - changing padding_side from right to left
2023-10-15 17:26:06.436 | INFO     | src.models.load:verbose_change_param:18 - changing truncation_side from right to left


In [4]:
ds_name = cfg.datasets[0]
split_type = "train"
ds_tokens = load_preproc_dataset(ds_name, cfg, tokenizer)

ds_tokens_calibration = ds_tokens.select(range(10))


Generating train split: 0 examples [00:00, ? examples/s]

Extracting 11 variants of each prompt


tokenize:   0%|          | 0/32 [00:00<?, ? examples/s]

truncated:   0%|          | 0/32 [00:00<?, ? examples/s]

prompt_truncated:   0%|          | 0/32 [00:00<?, ? examples/s]

choice_ids:   0%|          | 0/32 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32 [00:00<?, ? examples/s]

removed truncated rows to leave: num_rows 10


In [5]:
# b = next(iter(ds_tokens))
# b


In [6]:
# get the calibration dataset
BATCH_SIZE = 2
f = None
info_kwargs = dict(extract_cfg=cfg.to_dict(), ds_name=ds_name, split_type=split_type, f=f, date=pd.Timestamp.now().isoformat(),)
intervention_dicts = [None, ]
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds_tokens,
    batch_size=BATCH_SIZE,
    layer_padding=cfg.layer_padding,
    layer_stride=cfg.layer_stride,
    intervention_dicts=intervention_dicts,
)


In [24]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=json.dumps(info_kwargs, indent=2),
        config_name=f,
    ),
    gen_kwargs=gen_kwargs,
    num_proc=1,
)
ds1


Generating train split: 0 examples [00:00, ? examples/s]

get hidden states:   0%|          | 0/5 [00:00<?, ?it/s]

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
ds1.info.description
ds1['layer_names']


['model.layers.4.self_attn',
 'model.layers.8.self_attn',
 'model.layers.4.self_attn',
 'model.layers.8.self_attn',
 'model.layers.4.self_attn',
 'model.layers.8.self_attn',
 'model.layers.4.self_attn',
 'model.layers.8.self_attn',
 'model.layers.4.self_attn',
 'model.layers.8.self_attn']

# Scratch


In [8]:
from src.datasets.hs import ExtractHiddenStates
ehs = ExtractHiddenStates(model, tokenizer, intervention_dicts=intervention_dicts, layer_stride=cfg.layer_stride, layer_padding=cfg.layer_padding)
ehs


ExtractHiddenStates(model=LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=5120, out_features=32001, bias=False)
), tokenizer=LlamaTokenizerFast(name_or_path='TheBloke/WizardCoder-Python-13B-V1.0-GPTQ', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left',

In [9]:
from torch.utils.data import DataLoader
batch_size = BATCH_SIZE
data = ds_tokens_calibration
from src.helpers.ds import ds_keep_cols, clear_mem

# get a batch
torch_cols = ['input_ids', 'attention_mask', 'choice_ids']
ds_t_subset = ds_keep_cols(data, torch_cols)
ds_t_subset.set_format(type='torch')

ds_p_subset = data.remove_columns(torch_cols)
dl = DataLoader(ds_t_subset, batch_size=batch_size, shuffle=False)
for i, batch in enumerate(tqdm(dl, desc='get hidden states')):
    input_ids, attention_mask, choice_ids =  batch["input_ids"], batch["attention_mask"], batch["choice_ids"]
    
#     nn = len(input_ids)
#     index = i*batch_size+np.arange(nn)
    
#     # different due to dropout
#     hsl = ehs.get_batch_of_hidden_states(input_ids=input_ids, attention_mask=attention_mask, choice_ids=choice_ids)


get hidden states:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# from src.datasets.intervene import get_com_directions, get_interventions_dict
# head_wise_activations = np.array(ds1['head_activation'])
# labels=np.array(ds1["label_true"])
# get_com_directions(2, 
#                    2, 
#                    head_wise_activations, 
#                    labels
#                    )


In [11]:
from einops import rearrange, reduce, repeat, asnumpy, parse_shape
from src.datasets.intervene import InterventionDict
from typing import Tuple
from functools import partial
from baukit.nethook import Trace, TraceDict, recursive_copy
from src.datasets.intervene import intervention_meta_fn, get_interventions_dict

activations = np.array(ds1['head_activation']).squeeze(-1)
labels = np.array(ds1["label_true"]).astype(int)==1
num_heads = model.config.num_attention_heads

layer_names = [f"model.layers.{i}.self_attn" for i in range(model.config.num_hidden_layers)]
layer_names, layer_inds = ehs.get_layer_selection(layer_names)


In [12]:
ds1.info.description


'{\n  "extract_cfg": {\n    "datasets": [\n      "amazon_polarity",\n      "super_glue:boolq",\n      "glue:qnli",\n      "imdb"\n    ],\n    "model": "TheBloke/WizardCoder-Python-13B-V1.0-GPTQ",\n    "data_dirs": [],\n    "max_examples": [\n      10,\n      10\n    ],\n    "num_shots": 1,\n    "num_variants": -1,\n    "layers": [],\n    "seed": 42,\n    "token_loc": "last",\n    "template_path": null,\n    "max_length": 999\n  },\n  "ds_name": "amazon_polarity",\n  "split_type": "train",\n  "f": null,\n  "date": "2023-10-15T17:26:26.289433"\n}'

In [13]:


interventions = get_interventions_dict(activations, labels, layer_names, num_heads)
intervention_fn = partial(intervention_meta_fn, interventions=interventions, num_heads=num_heads)
model.cuda().eval()

device = model.device
with torch.no_grad():
    with TraceDict(model, layer_names, edit_output=intervention_fn) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4297, 12.6875,  0.4692,  ..., -2.2012, -1.2109, -2.1602],
         [-3.4297, 12.6875,  0.4695,  ..., -2.2012, -1.2129, -2.1621],
         [-3.4336, 12.6875,  0.4702,  ..., -2.2051, -1.2129, -2.1621],
         ...,
         [-2.4434,  1.2295, 11.8359,  ..., -2.0957,  0.1870,  0.2942],
         [-4.2383, -4.3594, 12.7656,  ..., -2.8594, -1.1748, -0.6260],
         [-6.4336, -6.3516, 11.0547,  ..., -5.6250, -2.6367, -2.8652]],

        [[-3.4062, 12.5859,  0.4016,  ..., -2.1797, -1.2871, -2.1543],
         [-3.4082, 12.5625,  0.3992,  ..., -2.1777, -1.2900, -2.1543],
         [-3.4062, 12.5703,  0.3999,  ..., -2.1777, -1.2910, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-7.2188, -8.3750,  9.3047,  ..., -5.4141, -2.8828, -3.3223]]],
       device='cuda:0')

In [14]:
with torch.no_grad():
    with TraceDict(model, layer_names) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4277, 12.6953,  0.4692,  ..., -2.2012, -1.2100, -2.1602],
         [-3.4297, 12.6875,  0.4695,  ..., -2.2012, -1.2119, -2.1602],
         [-3.4297, 12.6953,  0.4705,  ..., -2.2031, -1.2119, -2.1602],
         ...,
         [-2.4434,  1.2295, 11.8359,  ..., -2.0957,  0.1870,  0.2942],
         [-4.2383, -4.3594, 12.7656,  ..., -2.8594, -1.1748, -0.6260],
         [-5.4531, -4.9375, 10.1641,  ..., -4.8320, -2.1133, -2.2773]],

        [[-3.4043, 12.5703,  0.4009,  ..., -2.1777, -1.2881, -2.1523],
         [-3.4043, 12.5781,  0.4006,  ..., -2.1758, -1.2881, -2.1523],
         [-3.4043, 12.5703,  0.4006,  ..., -2.1777, -1.2900, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-6.3906, -7.0547,  8.7266,  ..., -4.8984, -2.3691, -2.8887]]],
       device='cuda:0')

In [15]:
intervention_fn2 = partial(intervention_meta_fn, interventions=interventions, num_heads=num_heads, alpha=-15)
model.cuda().eval()

device = model.device
with torch.no_grad():
    with TraceDict(model, layer_names, edit_output=intervention_fn2) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4277, 12.6953,  0.4695,  ..., -2.2012, -1.2090, -2.1602],
         [-3.4277, 12.6875,  0.4697,  ..., -2.1992, -1.2119, -2.1602],
         [-3.4316, 12.6875,  0.4705,  ..., -2.2051, -1.2119, -2.1621],
         ...,
         [-2.4434,  1.2295, 11.8359,  ..., -2.0957,  0.1870,  0.2942],
         [-4.2383, -4.3594, 12.7656,  ..., -2.8594, -1.1748, -0.6260],
         [-4.9805, -4.4336,  8.6094,  ..., -4.4102, -1.9639, -2.0254]],

        [[-3.4062, 12.5859,  0.4014,  ..., -2.1797, -1.2881, -2.1543],
         [-3.4082, 12.5859,  0.4011,  ..., -2.1797, -1.2881, -2.1562],
         [-3.4062, 12.5703,  0.3999,  ..., -2.1777, -1.2910, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-5.8555, -6.3125,  7.6172,  ..., -4.5625, -2.0273, -2.6055]]],
       device='cuda:0')